<a href="https://colab.research.google.com/github/BChun11/DATA3001/blob/main/DATA3001_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
# Download all the files into google colab environment
!git clone https://github.com/nokuik/KDDI-IoT-2019.git

Cloning into 'KDDI-IoT-2019'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 42 (delta 1), reused 9 (delta 1), pack-reused 33
Receiving objects: 100% (42/42), 776.84 MiB | 22.36 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (31/31), done.


In [4]:
# List all '.tar.gz files in ipfix directory
%cd KDDI-IoT-2019
%cd ipfix
!ls *.tar.gz

/content/KDDI-IoT-2019
/content/KDDI-IoT-2019/ipfix
 amazon_echo_gen2.tar.gz		        nature_remo.tar.gz
 au_network_camera.tar.gz		        panasonic_doorphone.tar.gz
 au_wireless_adapter.tar.gz		        philips_hue_bridge.tar.gz
 bitfinder_awair_breathe_easy.tar.gz	       'planex_camera_one_shot!.tar.gz'
 candy_house_sesami_wi-fi_access_point.tar.gz   planex_smacam_outdoor.tar.gz
 irobot_roomba.tar.gz			        planex_smacam_pantilt.tar.gz
 jvc_kenwood_cu-hb1.tar.gz		        powerelectric_wi-fi_plug.tar.gz
 jvc_kenwood_hdtv_ip_camera.tar.gz	        qrio_hub.tar.gz
 line_clova_wave.tar.gz			        sony_network_camera.tar.gz
 link_japan_eremote.tar.gz		        sony_smart_speaker.tar.gz
 mouse_computer_room_hub.tar.gz		        xiaomi_mijia_led.tar.gz


In [5]:
# List all tar.gz files and store them in a variable
files = !ls -1 *.tar.gz

# Extract each tar.gz file
for file in files:
    print(f"Extract {file} ")
    !tar -xzvf {file}

Extract amazon_echo_gen2.tar.gz 
amazon_echo_gen2.json
Extract au_network_camera.tar.gz 
au_network_camera.json
Extract au_wireless_adapter.tar.gz 
au_wireless_adapter.json
Extract bitfinder_awair_breathe_easy.tar.gz 
bitfinder_awair_breathe_easy.json
Extract candy_house_sesami_wi-fi_access_point.tar.gz 
candy_house_sesami_wi-fi_access_point.json
Extract irobot_roomba.tar.gz 
irobot_roomba.json
Extract jvc_kenwood_cu-hb1.tar.gz 
jvc_kenwood_cu-hb1.json
Extract jvc_kenwood_hdtv_ip_camera.tar.gz 
jvc_kenwood_hdtv_ip_camera.json
Extract line_clova_wave.tar.gz 
line_clova_wave.json
Extract link_japan_eremote.tar.gz 
link_japan_eremote.json
Extract mouse_computer_room_hub.tar.gz 
mouse_computer_room_hub.json
Extract nature_remo.tar.gz 
nature_remo.json
Extract panasonic_doorphone.tar.gz 
panasonic_doorphone.json
Extract philips_hue_bridge.tar.gz 
philips_hue_bridge.json
Extract 'planex_camera_one_shot!.tar.gz' 
planex_camera_one_shot!.json
Extract planex_smacam_outdoor.tar.gz 
planex_smacam

In [6]:
# Print the current working directory
print("Current Working Directory:", os.getcwd())

# List the contents of the current working directory
print("Contents of Current Directory:", os.listdir())

!cd

Current Working Directory: /content/KDDI-IoT-2019/ipfix
Contents of Current Directory: ['au_network_camera.tar.gz', 'qrio_hub.tar.gz', 'irobot_roomba.tar.gz', 'nature_remo.json', 'i-o_data_qwatch.tar.gz01', 'qrio_hub.json', 'planex_smacam_pantilt.json', 'amazon_echo_gen2.json', 'powerelectric_wi-fi_plug.json', 'irobot_roomba.json', 'au_wireless_adapter.json', 'sony_bravia.tar.gz00', 'panasonic_doorphone.tar.gz', 'au_network_camera.json', 'sony_bravia.tar.gz01', 'amazon_echo_gen2.tar.gz', 'bitfinder_awair_breathe_easy.tar.gz', 'nature_remo.tar.gz', 'philips_hue_bridge.tar.gz', 'bitfinder_awair_breathe_easy.json', 'planex_smacam_pantilt.tar.gz', 'xiaomi_mijia_led.json', 'jvc_kenwood_hdtv_ip_camera.tar.gz', 'link_japan_eremote.json', 'jvc_kenwood_hdtv_ip_camera.json', 'planex_smacam_outdoor.json', 'philips_hue_bridge.json', 'sony_bravia.tar.gz02', 'candy_house_sesami_wi-fi_access_point.tar.gz', 'mouse_computer_room_hub.json', 'mouse_computer_room_hub.tar.gz', 'sony_network_camera.json', '

In [7]:
# Code to generate distinct tables for each json file using a limited subset

# Define the directory where the JSON files are located
json_directory = '/content/KDDI-IoT-2019/ipfix'

# Get the list of all JSON files in the directory
json_files = [f for f in os.listdir(json_directory) if f.endswith('.json')]

# Create distinct tables for each json file
tables = {}
for json_file in json_files:
    # strip .json suffix from device names
    device_name = json_file.split('.')[0]
    # Construct the full path to the JSON file
    json_path = os.path.join(json_directory, json_file)
    # Read the JSON file into a DataFrame, normalize the 'flows' column, and get the first 1000 rows
    df = pd.json_normalize(pd.read_json(json_path, lines=True, nrows=1000)['flows'])

    # Label the DataFrame with the device name
    df['Device'] = device_name
    tables[device_name] = df

# Concatenate all the Dataframes in the tables dictionary into a single Dataframe
df = pd.concat(tables.values(), ignore_index=True)
df.head()

,flowStartMilliseconds,flowEndMilliseconds,flowDurationMilliseconds,reverseFlowDeltaMilliseconds,protocolIdentifier,sourceIPv4Address,sourceTransportPort,packetTotalCount,octetTotalCount,flowAttributes,...,reverseNonEmptyPacketCount,reverseDataByteCount,reverseAverageInterarrivalTime,reverseFirstNonEmptyPacketSize,reverseLargePacketCount,reverseMaxPacketSize,reverseStandardDeviationPayloadLength,reverseStandardDeviationInterarrivalTime,reverseBytesPerPacket,Device
0,2019-06-25 08:36:47.096,2019-06-25 08:36:47.675,0.579,0.0,17,192.168.1.1,39396,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo
1,2019-06-25 08:37:49.758,2019-06-25 08:37:50.337,0.579,0.0,17,192.168.1.1,58659,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo
2,2019-06-25 08:38:52.473,2019-06-25 08:38:53.028,0.555,0.0,17,192.168.1.1,60839,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo
3,2019-06-25 08:39:54.925,2019-06-25 08:39:55.509,0.584,0.0,17,192.168.1.1,55258,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo
4,2019-06-25 08:40:57.923,2019-06-25 08:40:58.502,0.579,0.0,17,192.168.1.1,51506,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo


In [8]:
# Convert variable flowStartMilliseconds & flowEndMilliseconds into datetime type
df['flowStartMilliseconds'] = pd.to_datetime(df['flowStartMilliseconds'])
df['flowEndMilliseconds'] = pd.to_datetime(df['flowEndMilliseconds'])

In [8]:
#df.set_index('flowStartMilliseconds', inplace=True)
#df.sort_index(inplace=True)

In [9]:
# reference : https://www.kaggle.com/code/arjanso/reducing-dataframe-memory-size-by-65 @ARJANGROEN

"""Reduce Memory Usage
  This code is used to reduce memory usage of our dataframe useful when running
  large datasets by reducing the size of the properties of the dataset by selecting
  smaller datatypes
"""

def reduce_memory_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        if ((col_type != 'datetime64[ns]') & (col_type != 'category')):
            if (col_type != 'object'):
                c_min = df[col].min()
                c_max = df[col].max()

                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        pass
            else:
                df[col] = df[col].astype('category')
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage became: ",mem_usg," MB")

    return df

In [55]:
# Apply the reduce memory usage function to our dataset
df = reduce_memory_usage(df)
df.info()

Memory usage of dataframe is 9.23 MB
Memory usage became:  2.7350540161132812  MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 55 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   flowStartMilliseconds                     22000 non-null  datetime64[ns]
 1   flowEndMilliseconds                       22000 non-null  datetime64[ns]
 2   flowDurationMilliseconds                  22000 non-null  float16       
 3   reverseFlowDeltaMilliseconds              22000 non-null  float16       
 4   protocolIdentifier                        22000 non-null  int8          
 5   sourceIPv4Address                         22000 non-null  category      
 6   sourceTransportPort                       22000 non-null  int32         
 7   packetTotalCount                          22000 non-null  int16         
 8   octetTotalCount         

In [10]:
df.head()

,flowStartMilliseconds,flowEndMilliseconds,flowDurationMilliseconds,reverseFlowDeltaMilliseconds,protocolIdentifier,sourceIPv4Address,sourceTransportPort,packetTotalCount,octetTotalCount,flowAttributes,...,reverseNonEmptyPacketCount,reverseDataByteCount,reverseAverageInterarrivalTime,reverseFirstNonEmptyPacketSize,reverseLargePacketCount,reverseMaxPacketSize,reverseStandardDeviationPayloadLength,reverseStandardDeviationInterarrivalTime,reverseBytesPerPacket,Device
0,2019-06-25 08:36:47.096,2019-06-25 08:36:47.675,0.579,0.0,17,192.168.1.1,39396,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo
1,2019-06-25 08:37:49.758,2019-06-25 08:37:50.337,0.579,0.0,17,192.168.1.1,58659,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo
2,2019-06-25 08:38:52.473,2019-06-25 08:38:53.028,0.555,0.0,17,192.168.1.1,60839,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo
3,2019-06-25 08:39:54.925,2019-06-25 08:39:55.509,0.584,0.0,17,192.168.1.1,55258,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo
4,2019-06-25 08:40:57.923,2019-06-25 08:40:58.502,0.579,0.0,17,192.168.1.1,51506,2,156,01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nature_remo


In [11]:
# Separate the Device column and flowStartMilliseconds as labels
labels_df = df[['flowStartMilliseconds', 'Device']].copy()

# Drop the Device column from the original DataFrame
df = df.drop(columns=['Device'])

In [12]:
labels_df.head()

,flowStartMilliseconds,Device
0,2019-06-25 08:36:47.096,nature_remo
1,2019-06-25 08:37:49.758,nature_remo
2,2019-06-25 08:38:52.473,nature_remo
3,2019-06-25 08:39:54.925,nature_remo
4,2019-06-25 08:40:57.923,nature_remo


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

In [14]:
"""
  Encode categorical values since Random Forest can't handle non-numeric
  categorical data directly
"""
# Identify Categorical Columns
categorical_col = []
for column in df.columns:
    if df[column].dtype == 'category' and len(df[column].unique()) <= 50:
        categorical_col.append(column)

label = LabelEncoder()
for column in categorical_col:
    df[column] = label.fit_transform(df[column])

In [15]:
X = df
y = labels_df

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Applying the Decision Tree and Random Forest Algorithm

Assessing the performance of classifier

In [16]:
"""
  Prints the accuracy, classification report, and confusion matrix for a classifier on the training or testing data.
  :param cls: Classifier to evaluate
  :param X_train: Training data features
  :param y_train: True labels for training data
  :param X_test: Testing data features
  :param y_test: True labels for testing data
  :param train: Boolean indicating whether to evaluate on the training or testing data
"""
def print_score(cls, X_train, y_train, X_test, y_test, train=True):
    # Check whether we are using the train or test data for evaluation
    if train:
        X, y, data_type = X_train, y_train, "Train"
    else:
        X, y, data_type = X_test, y_test, "Test"

    # The model 'cls' uses the features X to make predictions
    pred = cls.predict(X)
    # Generate reports for performance metrics (Accuracy Score, Classification Report and Confusion Matrix)
    cls_report = pd.DataFrame(classification_report(y, pred, output_dict=True))

    # Print results
    print(f"{data_type} Result:\n{'='*48}")
    print(f"Accuracy Score: {accuracy_score(y, pred) * 100:.2f}%")
    print("_______________________________________________")
    print(f"CLASSIFICATION REPORT:\n{clf_report}")
    print("_______________________________________________")
    print(f"Confusion Matrix: \n {confusion_matrix(y, pred)}\n")
